<a href="https://colab.research.google.com/github/chxbim/FP_ML_TomatoLeafDisease/blob/main/FP_ML_TomatoLeafDisease_Pertanian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import os
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.preprocessing import LabelEncoder
import pickle

In [17]:
base_dir = "/content/drive/MyDrive/dataset FP ML"
train_dir = os.path.join(base_dir, "train")
val_dir   = os.path.join(base_dir, "val")

#============================================check qty files train/val
for cls in os.listdir(val_dir):
    cls_path = os.path.join(val_dir, cls)
    n_files = len(os.listdir(cls_path))
    print(f"{cls}: {n_files} files")

for cls in os.listdir(train_dir):
    cls_path = os.path.join(train_dir, cls)
    n_files = len(os.listdir(cls_path))
    print(f"{cls}: {n_files} files")

Tomato___healthy: 100 files
Tomato___Tomato_mosaic_virus: 100 files
Tomato___Septoria_leaf_spot: 100 files
Tomato___Late_blight: 100 files
Tomato___Spider_mites Two-spotted_spider_mite: 100 files
Tomato___Target_Spot: 100 files
Tomato___Leaf_Mold: 100 files
Tomato___Early_blight: 100 files
Tomato___Tomato_Yellow_Leaf_Curl_Virus: 100 files
Tomato___Bacterial_spot: 100 files
Tomato___Target_Spot: 1000 files
Tomato___Late_blight: 1000 files
Tomato___Septoria_leaf_spot: 1000 files
Tomato___Tomato_Yellow_Leaf_Curl_Virus: 1000 files
Tomato___Bacterial_spot: 1000 files
Tomato___Early_blight: 1000 files
Tomato___Leaf_Mold: 1000 files
Tomato___Tomato_mosaic_virus: 1000 files
Tomato___Spider_mites Two-spotted_spider_mite: 510 files
Tomato___healthy: 0 files


In [18]:
IMG_SIZE = (224, 224)

#============================================load pretrained model (extractor)
base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')

def extract_feature(img_path):
    img = load_img(img_path, target_size=IMG_SIZE)
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    feat = base_model.predict(img, verbose=0)
    return feat.flatten()

def extract_dataset_features(root_dir):
    features = []
    labels = []
    classes = sorted(os.listdir(root_dir))

    for cls in classes:
        cls_path = os.path.join(root_dir, cls)
        if not os.path.isdir(cls_path):
            continue
        print(f"Processing class: {cls}")

        for img_name in tqdm(os.listdir(cls_path)):
            img_path = os.path.join(cls_path, img_name)
            feat = extract_feature(img_path)
            features.append(feat)
            labels.append(cls)

    return np.array(features), np.array(labels)


/tmp/ipython-input-4209194113.py:4: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')


In [19]:
x_train, y_train = extract_dataset_features(train_dir)
x_val, y_val = extract_dataset_features(val_dir)

encoder = LabelEncoder()
y_train_enc = encoder.fit_transform(y_train)
y_val_enc   = encoder.transform(y_val)

np.save("X_train.npy", X_train)
np.save("X_val.npy", X_val)
np.save("y_train.npy", y_train_enc)
np.save("y_val.npy", y_val_enc)

with open("label_encoder.pkl", "wb") as f:
    pickle.dump(encoder, f)

print("Saved all features!")


Processing class: Tomato___Bacterial_spot


  5%|▌         | 50/1000 [00:21<06:42,  2.36it/s]


KeyboardInterrupt: 